10 Exercise Monte Carlo methods: ***Frozen Lake***
==============

**Author**: `Hansruedi Keller, 14.01.2022`

### Part 0: Explore *Frozen Lake* Environment

In [1]:
!pip install gym

In [2]:
import sys
import gym
import numpy as np
from collections import defaultdict

In [3]:
# ======== ADJUST HERE AS APPROPRIATE ========
# env = gym.make('FrozenLake-v0')
env = gym.make('FrozenLake-v1', is_slippery=False)
env.render()                     # What does the board look like


SFFF
FHFH
FFFH
HFFG


In [4]:
print(env.observation_space)    # How many states are there?
print(env.action_space)         # How many actions are there?

Discrete(16)
Discrete(4)


---
**16 States**
*   SFFF       (S: starting point, safe)
*   FHFH       (F: frozen surface, safe)
*   FFFH       (H: hole, fall to your doom)
*   HFFG       (G: goal, where the frisbee is located)
---

**4 Actions**
*   0	Move Left
*   1	Move Down
*   2	Move Right
*   3	Move Up

In [5]:
# Show examples based on a random policy
for i_episode in range(3):
    state = env.reset()
    while True:
        # print(state)
        action = env.action_space.sample()
        state, reward, done, info = env.step(action)
        env.render();
        if done:
            print('End of Frozen Lake game! Reward: ', reward)
            if reward == 1:
              print('You reached the goal :)\n')
              break
            if state == 5 or state == 7 or state == 11 or state == 12:
               print('You fell into hole {} :-('.format(state))
               break
            print('You went too long, you will never find it. :-(')
            break
    print()

  (Up)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
End of Frozen Lake game! Reward:  0.0
You fell into hole 11 :-(

  (Up)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH

### Part 1: MC Prediction

The function accepts as **input**:
- `fl_env`: This is an instance of OpenAI Gym's Frozen Lake environment.

It returns as **output**:
- `episode`: This is a list of (state, action, reward) tuples (of tuples) and corresponds to $(S_0, A_0, R_1, \ldots, S_{T-1}, A_{T-1}, R_{T})$, where $T$ is the final time step.  In particular, `episode[i]` returns $(S_i, A_i, R_{i+1})$, and `episode[i][0]`, `episode[i][1]`, and `episode[i][2]` return $S_i$, $A_i$, and $R_{i+1}$, respectively.

In [6]:
def generate_episode_from_limit_stochastic(fl_env):
    episode = []
    state = fl_env.reset()
    while True:
        # Define probabilities =================================================
        probs =[[0.5, 0.25, 0.0, 0.25], [0.25, 0.5, 0.25, 0.0], [0.0, 0.25, 0.5, 0.25], [0.25, 0.0, 0.25, 0.5]] 
        # Define actions =======================================================
        action_choice = np.random.choice(np.arange(4))
        action = np.random.choice(np.arange(4), p=probs[action_choice])
        next_state, reward, done, info = fl_env.step(action)
        # ======== ADJUST HERE AS APPROPRIATE ========
        # print("State: {}, Action taken: {}, Next state: {}, Reward: {}".format(state, action, next_state, reward))
        # Record the walk
        episode.append((state, action, reward))
        state = next_state
        if done:
            # ======== ADJUST HERE AS APPROPRIATE ========
            # print('End of Frozen Lake tour')
            if reward == 1:
              print('!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
              print('Congratulation! You reached the goal :-)')
              print('!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
            break
    return episode

In [7]:
# Illustrate the procedure
for i in range(2):
    print("Episode =", i)
    print(generate_episode_from_limit_stochastic(env))
    print()

Episode = 0
[(0, 2, 0.0), (1, 3, 0.0), (1, 2, 0.0), (2, 0, 0.0), (1, 2, 0.0), (2, 3, 0.0), (2, 3, 0.0), (2, 0, 0.0), (1, 3, 0.0), (1, 2, 0.0), (2, 2, 0.0), (3, 3, 0.0), (3, 2, 0.0), (3, 1, 0.0)]

Episode = 1
[(0, 3, 0.0), (0, 3, 0.0), (0, 1, 0.0), (4, 2, 0.0)]



 Implementation of MC prediction with  arguments:
- `env`: This is an instance of an OpenAI Gym environment.
- `num_episodes`: This is the number of episodes that are generated through agent-environment interaction.
- `generate_episode`: This is a function that returns an episode of interaction.
- `gamma`: This is the discount rate.  It must be a value between 0 and 1, inclusive (default value: `1`).

The algorithm returns as output:
- `Q`: This is a dictionary (of one-dimensional arrays) where `Q[s][a]` is the estimated action value corresponding to state `s` and action `a`.

In [8]:
def mc_prediction_q(env, num_episodes, generate_episode, gamma=.99):
    # initialize empty dictionaries of arrays
    returns_sum = defaultdict(lambda: np.zeros(env.action_space.n))
    N = defaultdict(lambda: np.zeros(env.action_space.n))
    Q = defaultdict(lambda: np.zeros(env.action_space.n))
    # loop over episodes
    for i_episode in range(1, num_episodes+1):
        # monitor progress
        if i_episode % 1000 == 0:
            print("\rEpisode {}/{}.".format(i_episode, num_episodes), end="")
            sys.stdout.flush()
        # generate an episode (one game)
        # starting with own policy, we need a constant policy for prediction
        episode = generate_episode(env)
        # obtain the states, actions, and rewards. Compiles list of states, actions, rewards (tuples)
        states, actions, rewards = zip(*episode)
        # prepare for discounting (calculate list of gammas)
        discounts = np.array([gamma**i for i in range(len(rewards)+1)])
        
        # print("discounts:", discounts)
        # print("rewards:", rewards)
        # print("states:", states)
        # update the sum of the returns, number of visits, and action-value 
        # function estimates for each state-action pair in the episode
        # Update of Q-table afteer each episode (game)
        for i, state in enumerate(states):
            # print(i)
            # print("Rewards[i:]:", rewards[i:])
            # print("Discounts[:-(1+i)]:", discounts[:-(1+i)])
            # print("----------------------")
            # Next line is MOST IMPORTANT!
            returns_sum[state][actions[i]] += sum(rewards[i:]*discounts[:-(1+i)])
            N[state][actions[i]] += 1.0
            # MC prediction update, based on average
            Q[state][actions[i]] = returns_sum[state][actions[i]] / N[state][actions[i]]
    return Q

In [9]:
# Obtain the action-value function
# ======== ADJUST HERE AS APPROPRIATE ========
Q = mc_prediction_q(env, 1000, generate_episode_from_limit_stochastic)
V = dict((k,np.max(v)) for k, v in Q.items())
V

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Congratulation! You reached the goal :-)
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Congratulation! You reached the goal :-)
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Congratulation! You reached the goal :-)
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Congratulation! You reached the goal :-)
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Congratulation! You reached the goal :-)
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Congratulation! You reached the goal :-)
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Congratulation! You reached the goal :-)
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Congratulation! You reached the goal :-)
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
!!!!!!!!!!!!!!!!

{0: 0.019974824831129015,
 4: 0.02151158034602996,
 8: 0.06603272488220564,
 9: 0.1153352189215092,
 13: 0.5086480866545701,
 10: 0.48787094029642275,
 1: 0.04095017334201163,
 2: 0.07152093895205101,
 6: 0.23163155040873915,
 3: 0.011493181280673956,
 14: 1.0}

### Part 2: MC Control

In this section, you will write your own implementation of constant-$\alpha$ MC control.  

Your algorithm has four arguments:
- `env`: This is an instance of an OpenAI Gym environment.
- `num_episodes`: This is the number of episodes that are generated through agent-environment interaction.
- `alpha`: This is the step-size parameter for the update step.
- `gamma`: This is the discount rate.  It must be a value between 0 and 1, inclusive (default value: `1`).

The algorithm returns as output:
- `Q`: This is a dictionary (of one-dimensional arrays) where `Q[s][a]` is the estimated action value corresponding to state `s` and action `a`.
- `policy`: This is a dictionary where `policy[s]` returns the action that the agent chooses after observing state `s`.

(_Feel free to define additional functions to help you to organize your code._)

In [10]:
def generate_episode_from_Q(env, Q, epsilon, nA):
    """ generates an episode from following the epsilon-greedy policy """
    episode = []
    state = env.reset()
    while True:
        action = np.random.choice(np.arange(nA), p=get_probs(Q[state], epsilon, nA)) \
                                    if state in Q else env.action_space.sample()
        next_state, reward, done, info = env.step(action)
        episode.append((state, action, reward))
        state = next_state
        if done:
            break
    return episode

def get_probs(Q_s, epsilon, nA):                      # Policy evaluation
    """ obtains the action probabilities corresponding to epsilon-greedy policy """
    policy_s = np.ones(nA) * epsilon / nA
    best_a = np.argmax(Q_s)
    policy_s[best_a] = 1 - epsilon + (epsilon / nA)
    return policy_s

def update_Q(env, episode, Q, alpha, gamma):          # Policy improvement
    """ updates the action-value function estimate using the most recent episode """
    states, actions, rewards = zip(*episode)
    # Prepare for discounting
    discounts = np.array([gamma**i for i in range(len(rewards)+1)])
    for i, state in enumerate(states):
        old_Q = Q[state][actions[i]] 
        # Calculate returns and write these into matrix, starting with gammma^0 
        Q[state][actions[i]] = old_Q + alpha*(sum(rewards[i:]*discounts[:-(1+i)]) - old_Q)
    #
    # Q-values with index for state and index for action
    return Q

In [11]:
def mc_control(env, num_episodes, alpha, gamma=1.0, eps_start=1.0, eps_decay=0.999, eps_min=0.05):
    nA = env.action_space.n
    # initialize empty dictionary of arrays
    Q = defaultdict(lambda: np.zeros(nA))
    epsilon = eps_start
    # loop over episodes
    for i_episode in range(1, num_episodes+1):
        # monitor progress
        if i_episode % 1000 == 0:
            print("\rEpisode {}/{}.".format(i_episode, num_episodes), end="")
            sys.stdout.flush()
        # update the value of epsilon
        epsilon = max(epsilon*eps_decay, eps_min)
        # generate an episode by following epsilon-greedy policy
        episode = generate_episode_from_Q(env, Q, epsilon, nA)
        # update the action-value function estimate using the episode
        Q = update_Q(env, episode, Q, alpha, gamma)
    # determine the policy corresponding to the final action-value function estimate
    policy = dict((k,np.argmax(v)) for k, v in Q.items())
    return policy, Q

In [12]:
# ======== ADJUST HERE AS APPROPRIATE ========
# 1000+ episodes are required for successs
policy, Q = mc_control(env, 10000, 0.02)

Episode 10000/10000.

In [13]:
# Show state values V
V = dict((k,np.max(v)) for k, v in Q.items())
V

{0: 0.93329194353414,
 1: 0.9521786556553145,
 2: 0.9559570679604529,
 3: 0.7020679853132146,
 4: 0.9291282282393897,
 8: 0.9326343488953125,
 6: 0.9806641156445967,
 9: 0.9773055520319955,
 13: 0.9999999999999973,
 10: 0.9999999999999973,
 14: 0.9999999999999973}

In [14]:
# Show policy
policy

{0: 2, 1: 2, 2: 1, 3: 0, 4: 1, 8: 2, 6: 1, 9: 2, 13: 2, 10: 1, 14: 2}

### Part 3: Assess Success Rate

In [15]:
wins = 0
losses = 0
for i_episode in range(1000):
  state = env.reset()
  while state < 15:
    action = policy.get(state)
    # Show state and action to take
    # print(f'State, action {state, action}')
    state, reward, done, info = env.step(action)
    if done:
      if reward > 0:
        wins += 1
      else:
        losses += 1
      break
print(f'Success rate (wins): {wins/(wins+losses) *100}%')

Success rate (wins): 100.0%


With slippery = TRUE: Success rate (wins): 18.9% ... 27.6%